In [1]:
class FatBSTNode:
    def __init__(self, value=None, version=0, left=None, right=None):
        """Fat node with versioned left and right children and values."""
        self.value_versions = [(version, value)] if value is not None else []
        self.left_versions = [(version, left)]
        self.right_versions = [(version, right)]

    def get_at_version(self, versions_list, version):
        """Generic method to retrieve a versioned attribute."""
        for v, val in reversed(versions_list):
            if v <= version:
                return val
        return None

    def get_value(self, version):
        """Retrieve the value at a specific version."""
        return self.get_at_version(self.value_versions, version)

    def get_left(self, version):
        """Retrieve the left child at a specific version."""
        return self.get_at_version(self.left_versions, version)

    def get_right(self, version):
        """Retrieve the right child at a specific version."""
        return self.get_at_version(self.right_versions, version)

    def set_value(self, version, value):
        """Update the value at a new version."""
        self.value_versions.append((version, value))

    def set_left(self, version, left_node):
        """Update the left child at a new version."""
        self.left_versions.append((version, left_node))

    def set_right(self, version, right_node):
        """Update the right child at a new version."""
        self.right_versions.append((version, right_node))

    def clone_for_version(self, version):
        """Create a new node with the same properties at a new version."""
        new_node = FatBSTNode()
        new_node.set_value(version, self.get_value(version - 1))
        new_node.set_left(version, self.get_left(version - 1))
        new_node.set_right(version, self.get_right(version - 1))
        return new_node


class PersistentBST:
    def __init__(self):
        """Initializes a persistent BST with version tracking."""
        self.version = 0
        self.root_versions = [(self.version, None)]

    def get_root(self, version=None):
        """Retrieve the root of BST at a given version."""
        if version is None:
            version = self.version
        for v, root in reversed(self.root_versions):
            if v <= version:
                return root
        return None

    def _increment_version(self):
        """Helper to increment the version number."""
        self.version += 1
        return self.version

    def insert(self, value):
        """Insert a new value while keeping all previous versions intact."""
        version = self._increment_version()
        new_root = self._insert(self.get_root(version - 1), value, version)
        self.root_versions.append((version, new_root))
        return version

    def _insert(self, node, value, version):
        """Helper function to insert a node recursively and track versions."""
        if node is None:
            return FatBSTNode(value, version)

        new_node = node.clone_for_version(version)
        node_value = node.get_value(version - 1)

        if value < node_value:
            new_node.set_left(version, self._insert(node.get_left(version - 1), value, version))
        elif value > node_value:
            new_node.set_right(version, self._insert(node.get_right(version - 1), value, version))
        # If value already exists, we just return the clone without modification

        return new_node

    def search(self, value, version=None):
        """Search for a value in the BST at a given version."""
        if version is None:
            version = self.version
        return self._search(self.get_root(version), value, version)

    def _search(self, node, value, version):
        """Helper function to search recursively."""
        if node is None:
            return False
        node_value = node.get_value(version)
        if node_value == value:
            return True
        elif value < node_value:
            return self._search(node.get_left(version), value, version)
        else:
            return self._search(node.get_right(version), value, version)

    def delete(self, value):
        """Delete a value while maintaining history and create a new version."""
        version = self._increment_version()
        new_root = self._delete(self.get_root(version - 1), value, version)
        self.root_versions.append((version, new_root))
        return version

    def _delete(self, node, value, version):
        """Helper function to delete a node and track versioning."""
        if node is None:
            return None

        node_value = node.get_value(version - 1)

        if value < node_value:
            new_node = node.clone_for_version(version)
            new_node.set_left(version, self._delete(node.get_left(version - 1), value, version))
            return new_node
        elif value > node_value:
            new_node = node.clone_for_version(version)
            new_node.set_right(version, self._delete(node.get_right(version - 1), value, version))
            return new_node
        else:
            # Node to delete found
            left = node.get_left(version - 1)
            right = node.get_right(version - 1)

            if left is None:
                return right
            if right is None:
                return left

            # Find in-order successor (smallest node in right subtree)
            successor = self._find_min(right, version - 1)
            successor_value = successor.get_value(version - 1)

            # Create new node with successor's value
            new_node = node.clone_for_version(version)
            new_node.set_value(version, successor_value)
            # Remove the successor from the right subtree
            new_node.set_right(version, self._delete(right, successor_value, version))

            return new_node

    def _find_min(self, node, version):
        """Find the node with the smallest value in a subtree."""
        current = node
        while current.get_left(version) is not None:
            current = current.get_left(version)
        return current

    def in_order_traversal(self, version=None):
        """Return an in-order traversal list for a given version."""
        if version is None:
            version = self.version
        result = []
        self._in_order_traversal(self.get_root(version), version, result)
        return result

    def _in_order_traversal(self, node, version, result):
        """Helper function for in-order traversal."""
        if node is None:
            return
        self._in_order_traversal(node.get_left(version), version, result)
        result.append(node.get_value(version))
        self._in_order_traversal(node.get_right(version), version, result)

    def print_tree(self, version=None):
        """Print BST in an in-order traversal for a given version."""
        values = self.in_order_traversal(version)
        print(" ".join(map(str, values)))


# Example Usage
if __name__ == "__main__":
    pb_tree = PersistentBST()

    # Insert elements
    pb_tree.insert(50)
    pb_tree.insert(30)
    pb_tree.insert(70)
    pb_tree.insert(20)
    pb_tree.insert(40)
    pb_tree.insert(60)
    pb_tree.insert(80)

    # Print tree at latest version
    print("Tree at latest version:")
    pb_tree.print_tree()

    # Delete a node
    pb_tree.delete(30)
    print("Tree after deleting 30:")
    pb_tree.print_tree()

    # Access older version
    print("Tree at version 2:")
    pb_tree.print_tree(2)

    # Search for values
    print("Searching 30 at latest version:", pb_tree.search(30))
    print("Searching 30 at version 2:", pb_tree.search(30, 2))
    print("Searching 60 at latest version:", pb_tree.search(60))

Tree at latest version:
20 30 40 50 60 70 80
Tree after deleting 30:
20 40 50 60 70 80
Tree at version 2:
30 50
Searching 30 at latest version: False
Searching 30 at version 2: True
Searching 60 at latest version: True
